In [2]:
'''
用于对一次实验中的多路iFEMG信号进行特征提取

没有计算主动与拮抗肌肉之间的差值'''

import pandas as pd
import os

from gui_model_utils import form_feature_df
from iFEMG_feature import df_save_csv

In [32]:
"""
init dataframe
REVISE BEFORE RUNNING:  弯举/伸展动作切换时需要改
    1. ch_name_list
    2. ch_str
"""
feature_name_list = ['subject_name', 'height(cm)', 'weight(kg)', 'gender', 'age', 'label(kg)', 
                    'FMG', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']

ch1_str = "bicps_br"
ch2_str = "tricps_br_medial"
ch3_str = "tricps_br_lateral"

ch1_name_list = [ch1_str for i in feature_name_list]
ch2_name_list = [ch2_str for i in feature_name_list]
ch3_name_list = [ch3_str for i in feature_name_list]

all_feature_df = pd.DataFrame(columns=pd.MultiIndex.from_product([[ch1_str, ch2_str, ch3_str], feature_name_list]))
print(all_feature_df)
print("Init dataframe shape: ", all_feature_df.shape)

Empty DataFrame
Columns: [(bicps_br, subject_name), (bicps_br, height(cm)), (bicps_br, weight(kg)), (bicps_br, gender), (bicps_br, age), (bicps_br, label(kg)), (bicps_br, FMG), (bicps_br, mav), (bicps_br, rms), (bicps_br, wave_length), (bicps_br, zero_crossing), (bicps_br, slope_sign_change), (bicps_br, mean_freq), (bicps_br, mean_power_freq), (tricps_br_medial, subject_name), (tricps_br_medial, height(cm)), (tricps_br_medial, weight(kg)), (tricps_br_medial, gender), (tricps_br_medial, age), (tricps_br_medial, label(kg)), (tricps_br_medial, FMG), (tricps_br_medial, mav), (tricps_br_medial, rms), (tricps_br_medial, wave_length), (tricps_br_medial, zero_crossing), (tricps_br_medial, slope_sign_change), (tricps_br_medial, mean_freq), (tricps_br_medial, mean_power_freq), (tricps_br_lateral, subject_name), (tricps_br_lateral, height(cm)), (tricps_br_lateral, weight(kg)), (tricps_br_lateral, gender), (tricps_br_lateral, age), (tricps_br_lateral, label(kg)), (tricps_br_lateral, FMG), (tricps_

In [33]:
# signal_channel, 信号对应的传感器通道
signal_channel_dic = {"bicps_br": 1,
                      "tricps_br_lateral": 2,
                      "tricps_br_medial": 8}

# 存储文件名对应重量标签的dic
bicps_dic = {'bi-0': 0, 'bi-05': 0.5, 'bi-1': 1, 'bi-2': 2}
tricps_dic = {'tri-0': 0, 'tri-05': 0.5, 'tri-1': 1}

# 需要更改
folder_path = r"E:\Data\20230310-iFEMG肌力测试\zpk-2"
file_name_dic = tricps_dic
subject_name = "zpk"
subject_arg_input = {"subject_height": 190,
                    "subject_weight": 80,
                    "subject_age": 26,
                    "subject_gender": 1,
                    "subject_name": subject_name}

for key, value in file_name_dic.items():
    #raw_data = pd.read_table(folder_path+'\\'+file_name_list1[i]+'.db', sep = ';', header = None)
    #label = read_label(folder_path+'\\'+file_name_list1[i]+'.txt')
    temp_df = form_feature_df(db_file_path = folder_path + '\\' + key + '.db',
                              time_file_path = folder_path + '\\' + key + '.txt',
                              channel_name_dic = signal_channel_dic,
                              abandon_ms = 1000,
                              strength_level = value,
                              **subject_arg_input)
    try:
        print(temp_df.shape)
    except AttributeError:
        pass
    # 数据加入all_feature_df
    all_feature_df = pd.concat([all_feature_df, temp_df], axis=0, ignore_index=True)
    print(all_feature_df.shape)
    pass

all_feature_df.head


(5, 42)
(5, 42)
No such file: ('E:\\Data\\20230310-iFEMG肌力测试\\zpk-2\\tri-05.db', 'E:\\Data\\20230310-iFEMG肌力测试\\zpk-2\\tri-05.txt')
(5, 42)
(5, 42)
(10, 42)


<bound method NDFrame.head of       bicps_br                                                          \
  subject_name height(cm) weight(kg) gender age label(kg)          FMG   
0          zpk        190         80      1  26         0   617.078346   
1          zpk        190         80      1  26         0   947.145390   
2          zpk        190         80      1  26         0  1094.941240   
3          zpk        190         80      1  26         0  1013.833107   
4          zpk        190         80      1  26         0  1049.699918   
5          zpk        190         80      1  26         1   854.170712   
6          zpk        190         80      1  26         1   492.321311   
7          zpk        190         80      1  26         1   805.931880   
8          zpk        190         80      1  26         1   886.927362   
9          zpk        190         80      1  26         1   838.612620   

                                   ... tricps_br_lateral            \
        mav

In [34]:
"""
写入文件，增加覆盖检查功能
运行前更改：
    1. 肌肉名称
"""
filename = folder_path + '\\' + subject_name + "_tricps_br_absnew.csv"

df_save_csv(all_feature_df, filename)

File E:\Data\20230310-iFEMG肌力测试\zpk-2\zpk_tricps_br_absnew.csv saved!
